# Tank detection YoloV8 model train

This notebook will train a [Yolov8](https://github.com/ultralytics/ultralytics) model for tank detection using publicly available annotated images of tanks.

As the notebook will run the training with `PyTorch`, it is recommended to have GPUs available. If running in Google Colab, go to Edit > Notebook settings and select GPU hardware acceleration.

The first step of the notebook will use [fiftyone](https://github.com/voxel51/fiftyone), to build a dataset of tank images and bounding box annotations for training the computer vision model. The tutorial notebook on [Fine-tuning YOLOv8 models for custom use cases](https://github.com/voxel51/fiftyone/blob/v0.21.0/docs/source/tutorials/yolov8.ipynb) is a usefull introduction on how to use `fiftyone`.

### Setup
To get started, install FiftyOne and Ultralytics (Yolov8) and check PyTorch and GPU support.

In [1]:
!pip install fiftyone ultralytics

import torch
from IPython.display import clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.0.1+cu118 (Tesla T4)


We also setup some logging.

In [2]:
import logging
import os

logging.basicConfig(level=logging.INFO)
NOTEBOOK_DIR = os.path.abspath('')
LOG = logging.getLogger()
LOG.setLevel(logging.INFO)

### Download images from ImageNet21k

The first dataset we'll use is ImageNet21k. The ImageNet21k dataset is available at [https://image-net.org/download-images.php](https://image-net.org/download-images.php). You need to register and be granted access to download the images. We use the Winter 21 version since it gives the option of downloading the images for a single synset: https://image-net.org/data/winter21_whole/SYNSET_ID.tar, e.g., https://image-net.org/data/winter21_whole/n02352591.tar. The processed version of ImageNet21k is available here : https://github.com/Alibaba-MIIL/ImageNet21K. The classes ids and names are available here https://github.com/google-research/big_transfer/issues/7#issuecomment-640048775.

We'll begin by downloading the classe names that are in ImageNet21k and look for relevant classes that we can use.

In [3]:
import requests
import shutil
from pathlib import Path

def download_file(url: str, filename: Path) -> Path:
    if filename.exists():
        LOG.info(f'File {filename} already exists. Skipping download.')
    else:
        LOG.info(f'Downloading {filename} ...')
        with requests.get(url, stream=True) as r:
            with open(filename, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
        LOG.info('Download complete.')

In [4]:
from typing import Dict

def download_class_names() -> Dict[str, str]:
    download_dir = Path(NOTEBOOK_DIR)
    id_file = download_dir / 'imagenet21k_wordnet_ids.txt'
    name_file = download_dir / 'imagenet21k_wordnet_lemmas.txt'

    download_file('https://storage.googleapis.com/bit_models/imagenet21k_wordnet_ids.txt', id_file)
    download_file('https://storage.googleapis.com/bit_models/imagenet21k_wordnet_lemmas.txt', name_file) 
         
    with open(id_file, 'r') as f:
        ids = f.readlines()

    with open(name_file, 'r') as f:
        names = f.readlines()

    synsets = {ids[i].strip(): names[i].strip() for i in range(len(ids))}
    return synsets

In [5]:
synsets = download_class_names()

INFO:root:Downloading /content/imagenet21k_wordnet_ids.txt ...
INFO:root:Download complete.
INFO:root:Downloading /content/imagenet21k_wordnet_lemmas.txt ...
INFO:root:Download complete.


We can now search the synsets for relevant keywords, i.e.

- Lemma: **tank, army_tank, armored_combat_vehicle, armoured_combat_vehicle**; Class: n04389033
- Lemma: **armored_personnel_carrier, armoured_personnel_carrier, APC**; Class: n02740300
- Lemma: **armored_vehicle, armoured_vehicle**; Class: n02740533
- Lemma: **tracked_vehicle**; Class: n04464852
- Lemma: **military_vehicle**; Class: n03764276

In [6]:
import re

def find_class_by_text(synsets, query):
    for id, lemma in synsets.items():
        if re.search(query, lemma, re.IGNORECASE):
            print(f'Lemma: {lemma}; Class: {id}')

In [7]:
find_class_by_text(synsets, 'armored')

Lemma: armored_dinosaur; Class: n01701551
Lemma: armored_scale; Class: n02249515
Lemma: armored_catfish; Class: n02520525
Lemma: armored_car, armoured_car; Class: n02739889
Lemma: armored_car, armoured_car; Class: n02740061
Lemma: armored_personnel_carrier, armoured_personnel_carrier, APC; Class: n02740300
Lemma: armored_vehicle, armoured_vehicle; Class: n02740533
Lemma: tank, army_tank, armored_combat_vehicle, armoured_combat_vehicle; Class: n04389033


We can now download images and annotations for the relevant classes. The `download_imagenet_detections` function will download the images and annotations for the given synset ids **if the annotations exist** (not all classes have been annotated).

In [8]:
import tarfile
from typing import List

def download_annotations(class_ids: List[str], dataset_dir: str) -> List[str]:
    # Download zipfile with detections for all classes
    dataset_path = Path(NOTEBOOK_DIR) / dataset_dir
    annotations_file = dataset_path / "bboxes_annotations.tar.gz"
    annotations_dir = dataset_path / "bboxes_annotations"
    download_file('https://image-net.org/data/bboxes_annotations.tar.gz', annotations_file)

    # Extract annotations
    with tarfile.open(annotations_file, "r:gz") as tf:
        tf.extractall(annotations_dir)

    # Extract annotations for each class
    annoted_classes = []
    for class_id in class_ids:
        class_label_dir = dataset_path / "labels" / class_id
        if class_label_dir.exists():
            LOG.info(f'Annotations directory {class_label_dir} already exists. Skipping extract.')
        else:
            annotations_class_file = annotations_dir / f"{class_id}.tar.gz"
            if annotations_class_file.exists():
                with tarfile.open(annotations_class_file, "r:gz") as tf:
                    tf.extractall(annotations_dir)
                shutil.move(annotations_dir / "Annotation" / class_id, class_label_dir)
                LOG.info(f'Extracted annotations for {class_id} to {class_label_dir}')
                annoted_classes.append(class_id)
            else:
                LOG.info(f'There are not annotations for class {class_id}.')

    # Delete annotations directory
    LOG.info('Deleting annotations dir.')
    shutil.rmtree(annotations_dir)
    return annoted_classes

def download_imagenet_detections(class_ids: List[str], dataset_dir: str):
    # Create dataset_dir
    dataset_path = Path(NOTEBOOK_DIR) / dataset_dir
    dataset_path.mkdir(exist_ok=True)
    data_dir = dataset_path / "data"
    data_dir.mkdir(exist_ok=True)
    labels_dir = dataset_path / "labels"
    labels_dir.mkdir(exist_ok=True)

    annoted_classes = download_annotations(class_ids, dataset_dir)

    # Download synset images for each class with annotations
    for class_id in annoted_classes:
        class_dir = data_dir / class_id
        if class_dir.exists():
            LOG.info(f'Directory {class_dir} already exists. Skipping download.')
        else:
            tarfilename = dataset_path / f'{class_id}.tar'
            url = f'https://image-net.org/data/winter21_whole/{class_id}.tar'
            download_file(url, tarfilename)
            with tarfile.open(tarfilename) as tf:
                tf.extractall(class_dir)
            LOG.info(f'Extracted {class_dir}.')
    

In [9]:
dataset_dir = "imagenet"
classes = ["n02740300", "n04389033", "n02740533", "n04464852", "n03764276"]
download_imagenet_detections(classes, dataset_dir)

INFO:root:Downloading /content/imagenet/bboxes_annotations.tar.gz ...
INFO:root:Download complete.
INFO:root:There are not annotations for class n02740300.
INFO:root:Extracted annotations for n04389033 to /content/imagenet/labels/n04389033
INFO:root:There are not annotations for class n02740533.
INFO:root:There are not annotations for class n04464852.
INFO:root:There are not annotations for class n03764276.
INFO:root:Deleting annotations dir.
INFO:root:Downloading /content/imagenet/n04389033.tar ...
INFO:root:Download complete.
INFO:root:Extracted /content/imagenet/data/n04389033.


### Create a fiftyone dataset with the downloaded ImageNet data

Now that we're downloaded images and annotations, we can create a fiftyone dataset to manage it. The first step is to remove labels which have no corresonding image, as this causes errors when importing the data into fiftyone.

In [10]:
def cleanup_labels_without_images(dataset_dir: str):
    dataset_path = Path(NOTEBOOK_DIR) / dataset_dir
    data_dir = dataset_path / "data"
    labels_dir = dataset_path / "labels"
    classes = [path.name for path in data_dir.iterdir() if path.is_dir()]
    for class_id in classes:
        images = {path.stem for path in (data_dir / class_id).iterdir() if not path.is_dir()}
        labels = {path.stem for path in (labels_dir / class_id).iterdir() if not path.is_dir()}
        LOG.info(f'Deleting {len(labels.difference(images))} labels without images')
        for label_id in labels.difference(images):
            filename = labels_dir / class_id / (label_id + '.xml')
            filename.unlink()

In [11]:
cleanup_labels_without_images(dataset_dir)

INFO:root:Deleting 138 labels without images


We can now create a new dataset. Note that we set this dataset to be persistent, so you should use the `load_dataset('military-vehicles')` function to reload the dataset on ulterior runs of the notebook.

In [12]:
import fiftyone as fo

# Create the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=fo.types.VOCDetectionDataset,
)


dataset.name = "military-vehicles"
dataset.persistent = True

view = dataset.map_labels(
    "ground_truth",
    {"n04389033":"tank"}
)
view.save()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Migrating database to v0.21.0


INFO:fiftyone.migrations.runner:Migrating database to v0.21.0
INFO:httpx:HTTP Request: POST https://www.google-analytics.com/collect "HTTP/1.1 200 OK"


 100% |█████████████████| 378/378 [700.4ms elapsed, 0s remaining, 539.7 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 378/378 [700.4ms elapsed, 0s remaining, 539.7 samples/s]      


In [13]:
session = fo.launch_app(dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.21.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.21.0

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



In [14]:
session.freeze()

### Add OpenImage samples

The ImageNet dataset only contained 378 annotated images of tanks, so we'll look into other available datasets to improve training of the model. We’ll load [Open Images](https://storage.googleapis.com/openimages/web/index.html) samples with `Tank` detection labels, passing in `only_matching=True` to only load the `Tank` labels. We then map these labels by changing `Tank` into `tank`.

In [15]:
import fiftyone.zoo as foz

oi_samples = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Tank"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Tank":"tank"}
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [3.2s elapsed, 0s remaining, 1.7Gb/s]         


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [3.2s elapsed, 0s remaining, 1.7Gb/s]         


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/train/metadata/classes.csv'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 50/50 [116.8ms elapsed, 0s remaining, 459.8 samples/s]    


INFO:eta.core.utils: 100% |███████████████████| 50/50 [116.8ms elapsed, 0s remaining, 459.8 samples/s]    


Dataset 'open-images-v7' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7' created


We can add these new samples into our training dataset with `merge_samples()`:

In [16]:
dataset.merge_samples(oi_samples)

In [17]:
session = fo.launch_app(dataset)

In [18]:
session.freeze()

Our dataset now contains 1624 annotated images of tanks.

## Fine-tune a YOLOv8 detection model

Now that our dataset is created, we'll export it into a format supported by YOLOv8 to train our model.

In [19]:
import fiftyone.utils.random as four

# load dataset
dataset = fo.load_dataset("military-vehicles")

## delete existing tags to start fresh
dataset.untag_samples(dataset.distinct("tags"))

## split into train, test and val
four.random_split(
    dataset,
    {"train": 0.8, "val": 0.1, "test": 0.1}
)

The `export_yolo_data` function will export our dataset into the given directory.

In [20]:
def export_yolo_data(
    samples, 
    export_dir, 
    classes, 
    label_field = "ground_truth", 
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples, 
                export_dir, 
                classes, 
                label_field, 
                split
            )   
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

In [21]:
## export in YOLO format
export_yolo_data(
    dataset, 
    "vehicles", 
    ["tank"], 
    split = ["train", "val", "test"]
)

 100% |███████████████| 1299/1299 [6.2s elapsed, 0s remaining, 149.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1299/1299 [6.2s elapsed, 0s remaining, 149.0 samples/s]      


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 163/163 [563.3ms elapsed, 0s remaining, 289.3 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 163/163 [563.3ms elapsed, 0s remaining, 289.3 samples/s]      


Directory 'vehicles' already exists; export will be merged with existing files


 100% |█████████████████| 162/162 [532.4ms elapsed, 0s remaining, 304.3 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 162/162 [532.4ms elapsed, 0s remaining, 304.3 samples/s]      


Now all that is left is to do the fine-tuning! We will use YOLO command line syntax, with mode=train. We will specify the initial weights as the starting point for training, the number of epochs, image size, and batch size.

For this notebook, we use the `yolov8n.pt` (nano) model, which is the smallest, but larger models are available from ultralytics.

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=vehicles/dataset.yaml epochs=60 imgsz=640 batch=16

100% 6.23M/6.23M [00:00<00:00, 187MB/s]
Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=vehicles/dataset.yaml, epochs=60, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, 

Create a zip file with train results and download it:

In [ ]:
!zip -r yolov8_military.zip runs/detect/train/

  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/args.yaml (deflated 51%)
  adding: content/runs/detect/train/results.png (deflated 7%)
  adding: content/runs/detect/train/labels_correlogram.jpg (deflated 36%)
  adding: content/runs/detect/train/train_batch0.jpg (deflated 5%)
  adding: content/runs/detect/train/val_batch0_pred.jpg (deflated 10%)
  adding: content/runs/detect/train/F1_curve.png (deflated 17%)
  adding: content/runs/detect/train/confusion_matrix_normalized.png (deflated 38%)
  adding: content/runs/detect/train/events.out.tfevents.1686316091.2a7eb5a7cbe9.9783.0 (deflated 72%)
  adding: content/runs/detect/train/val_batch1_labels.jpg (deflated 6%)
  adding: content/runs/detect/train/results.csv (deflated 85%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 39%)
  adding: content/runs/detect/train/PR_curve.png (deflated 22%)
  adding: content/runs/detect/train/val_batch0_labels.jpg (deflated 11%)
  adding: content/runs/

In [ ]:
from google.colab import files
files.download('/content/yolov8_military.zip') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

With fine-tuning complete, we can generate predictions on our test data with the “best” weights found during the training process, which are stored at `runs/detect/train/weights/best.pt`:

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source=vehicles/images/test save_txt=True save_conf=True

Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Could not connect session, trying again in 10 seconds

Model summary (fused): 168 layers, 3005843 parameters, 0 gradients

image 1/162 /content/vehicles/images/test/0060fa461e2f119e.jpg: 448x640 2 tanks, 59.4ms
image 2/162 /content/vehicles/images/test/007377a82dc87b7f.jpg: 448x640 2 tanks, 6.3ms
image 3/162 /content/vehicles/images/test/011458ec5d8d61d8.jpg: 448x640 1 tank, 6.2ms
image 4/162 /content/vehicles/images/test/01c1bff29e25b149.jpg: 320x640 2 tanks, 53.8ms
image 5/162 /content/vehicles/images/test/01d94857a1fde861.jpg: 448x640 3 tanks, 6.7ms
image 6/162 /content/vehicles/images/test/021bb7e1cbb12d26.jpg: 384x640 1 tank, 59.9ms
image 7/162 /content/vehicles/images/test/03a3424b6d4591b6.jpg: 640x640 2 tanks, 7.9ms
image 8/162 /content/vehicles/images/test/07a684cf90b5df0e.jpg: 480x640 2 tanks, 52.9ms
image 9/162 /content/vehicles/images/test/07e79f408f1d4e90.jpg: 448x640 1 tank, 6.9ms
image

In [ ]:
# The test split of the dataset
test_view = dataset.match_tags("test")

### Load model detections

We can read a YOLOv8 detection prediction file with $N$ detections into an $(N, 6)$ numpy array:

In [ ]:
import numpy as np
from tqdm import tqdm

def read_yolo_detections_file(filepath):
    detections = []
    if not os.path.exists(filepath):
        return np.array([])
    
    with open(filepath) as f:
        lines = [line.rstrip('\n').split(' ') for line in f]
    
    for line in lines:
        detection = [float(l) for l in line]
        detections.append(detection)
    return np.array(detections)

From here, we need to convert these detections into FiftyOne’s [Detections](https://docs.voxel51.com/user_guide/using_datasets.html#object-detection) format.

YOLOv8 represents bounding boxes in a centered format with coordinates `[center_x, center_y, width, height]`, whereas [FiftyOne stores bounding boxes](https://docs.voxel51.com/user_guide/using_datasets.html#object-detection) in `[top-left-x, top-left-y, width, height]` format. We can make this conversion by "un-centering" the predicted bounding boxes:

In [ ]:
def _uncenter_boxes(boxes):
    '''convert from center coords to corner coords'''
    boxes[:, 0] -= boxes[:, 2]/2.
    boxes[:, 1] -= boxes[:, 3]/2.

Additionally, we can convert a list of class predictions (indices) to a list of class labels (strings) by passing in the class list:


In [ ]:
def _get_class_labels(predicted_classes, class_list):
    labels = (predicted_classes).astype(int)
    labels = [class_list[l] for l in labels]
    return labels

Given the output of a `read_yolo_detections_file()` call, `yolo_detections`, we can generate the FiftyOne `Detections` object that captures this data:

In [ ]:
def convert_yolo_detections_to_fiftyone(
    yolo_detections, 
    class_list
    ):

    detections = []
    if yolo_detections.size == 0:
        return fo.Detections(detections=detections)
    
    boxes = yolo_detections[:, 1:-1]
    _uncenter_boxes(boxes)
    
    confs = yolo_detections[:, -1]
    labels = _get_class_labels(yolo_detections[:, 0], class_list) 
 
    for label, conf, box in zip(labels, confs, boxes):
        detections.append(
            fo.Detection(
                label=label,
                bounding_box=box.tolist(),
                confidence=conf
            )
        )

    return fo.Detections(detections=detections)

The final ingredient is a function that takes in the file path of an image, and returns the file path of the corresponding YOLOv8 detection prediction text file.

In [ ]:
def get_prediction_filepath(filepath, run_number = 1):
    run_num_string = ""
    if run_number != 1:
        run_num_string = str(run_number)
    filename = filepath.split("/")[-1].split(".")[0]
    return f"runs/detect/predict{run_num_string}/labels/{filename}.txt"

If you run multiple inference calls for the same task, the predictions results are stored in a directory with the next available integer appended to `predict` in the file path. You can account for this in the above function by passing in the `run_number` argument.

Putting the pieces together, we can write a function that adds these YOLOv8 detections to all of the samples in our dataset efficiently by batching the read and write operations to the underlying [MongoDB database](https://docs.voxel51.com/environments/index.html#connecting-to-a-localhost-database).

In [ ]:
def add_yolo_detections(
    samples,
    prediction_field,
    prediction_filepath,
    class_list
    ):

    prediction_filepaths = samples.values(prediction_filepath)
    yolo_detections = [read_yolo_detections_file(pf) for pf in prediction_filepaths]
    detections =  [convert_yolo_detections_to_fiftyone(yd, class_list) for yd in yolo_detections]
    samples.set_values(prediction_field, detections)

Now we can rapidly add the detections in a few lines of code:

In [ ]:
filepaths = test_view.values("filepath")
prediction_filepaths = [get_prediction_filepath(fp) for fp in filepaths]
test_view.set_values(
    "yolov8n_det_filepath", 
    prediction_filepaths
)

add_yolo_detections(
    test_view, 
    "yolov8n", 
    "yolov8n_det_filepath", 
    ["tank"]
)


Could not connect session, trying again in 10 seconds



Now we can visualize these YOLOv8 model predictions on the samples in our dataset in the FiftyOne App:

In [ ]:
session = fo.launch_app(test_view)


Could not connect session, trying again in 10 seconds



In [ ]:
session.freeze()

### Evaluate model predictions

In [ ]:
detection_results = test_view.evaluate_detections(
    "yolov8n", 
    eval_key="eval",
    compute_mAP=True,
    gt_field="ground_truth",
)

Evaluating detections...


INFO:fiftyone.utils.eval.detection:Evaluating detections...


 100% |█████████████████| 162/162 [708.1ms elapsed, 0s remaining, 228.8 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 162/162 [708.1ms elapsed, 0s remaining, 228.8 samples/s]      


Performing IoU sweep...


INFO:fiftyone.utils.eval.coco:Performing IoU sweep...


 100% |█████████████████| 162/162 [819.9ms elapsed, 0s remaining, 197.6 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 162/162 [819.9ms elapsed, 0s remaining, 197.6 samples/s]      



Could not connect session, trying again in 10 seconds



In [ ]:
mAP = detection_results.mAP()
print(f"mAP = {mAP}")

mAP = 0.4973578627422652


In [ ]:
detection_results.print_report()

              precision    recall  f1-score   support

        tank       0.66      0.74      0.70       249

   micro avg       0.66      0.74      0.70       249
   macro avg       0.66      0.74      0.70       249
weighted avg       0.66      0.74      0.70       249

